Axial Force Calculation

In [1]:
from lib.tripping import trip_operation
from lib.circulation import circl
import numpy as np

# Define the range of total lengths
total_lengths = np.arange(12500, 18001, 100)
# total_lengths = [17000]
wob = 0

hook_POOH_470 = np.array([trip_operation(F_b=wob,  flowrate=470, length=i, operation='POOH')[-1] for i in total_lengths])
hook_RIH_470 = np.array([trip_operation(F_b=wob,  flowrate=470, length=i, operation='RIH')[-1] for i in total_lengths])
hook_ROB_470 = np.array([trip_operation(F_b=wob,  flowrate=470, length=i, operation='ROB')[-1] for i in total_lengths])
hook_POOH_250 = np.array([trip_operation(F_b=wob,  flowrate=250, length=i, operation='POOH')[-1] for i in total_lengths])
hook_RIH_250 = np.array([trip_operation(F_b=wob,  flowrate=250, length=i, operation='RIH')[-1] for i in total_lengths])
hook_ROB_250 = np.array([trip_operation(F_b=wob,  flowrate=250, length=i, operation='ROB')[-1] for i in total_lengths])
hook_POOH_0 = np.array([trip_operation(F_b=wob,  flowrate=0, length=i, operation='POOH')[-1] for i in total_lengths])
hook_RIH_0 = np.array([trip_operation(F_b=wob,  flowrate=0, length=i, operation='RIH')[-1] for i in total_lengths])
hook_ROB_0 = np.array([trip_operation(F_b=wob,  flowrate=0, length=i, operation='ROB')[-1] for i in total_lengths])

c:\Users\User\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Web Extension extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
total_lengths

TypeError: Cannot interpret '12600' as a data type

In [ ]:
from lib1.tripping import *
from lib1.circulation import circl

# Define the range of total lengths
# total_lengths = list(range(12500, 18001, 100))
total_lengths = [13000]
wob = 0

POOH_470 = [trip_POOH(wob, ff=0.15, length=i) - circl(flowrate=470, length=i) for i in total_lengths]
RIH_470 = [trip_RIH(wob, ff=0.15, length=i) - circl(flowrate=470, length=i) for i in total_lengths]
ROB_470 = [trip_POOH(wob, ff=0, length=i) - circl(flowrate=470, length=i) for i in total_lengths]
POOH_250 = [trip_POOH(wob, ff=0.15, length=i) - circl(flowrate=250, length=i) for i in total_lengths]
RIH_250 = [trip_RIH(wob, ff=0.15, length=i) - circl(flowrate=250, length=i) for i in total_lengths]
ROB_250 = [trip_POOH(wob, ff=0, length=i) - circl(flowrate=250, length=i) for i in total_lengths]
POOH_0 = [trip_POOH(wob, ff=0.15, length=i) for i in total_lengths]
RIH_0 = [trip_RIH(wob, ff=0.15, length=i) for i in total_lengths]
ROB_0 = [trip_POOH(wob, ff=0, length=i) for i in total_lengths]

In [ ]:
hook_POOH_470 = [i.sum() + wob for i in POOH_470]
hook_RIH_470 = [i.sum() + wob for i in RIH_470]
hook_ROB_470 = [i.sum() + wob for i in ROB_470]
hook_POOH_250 = [i.sum() + wob for i in POOH_250]
hook_RIH_250 = [i.sum() + wob for i in RIH_250]
hook_ROB_250 = [i.sum() + wob for i in ROB_250]
hook_POOH_0 = [i.sum() + wob for i in POOH_0]
hook_RIH_0 = [i.sum() + wob for i in RIH_0]
hook_ROB_0 = [i.sum() + wob for i in ROB_0]

With - sign

In [ ]:
hook_POOH_0, hook_RIH_0, hook_ROB_0

In [ ]:
hook_POOH_0, hook_RIH_0, hook_ROB_0

Without - sign

In [ ]:
hook_POOH_0, hook_RIH_0, hook_ROB_0

In [ ]:
hook_POOH_0, hook_RIH_0, hook_ROB_0

Hookload Results - 13000 ft

In [2]:
hook_POOH_0[0]+20000, hook_POOH_250[0]+20000, hook_POOH_470[0]+20000

(284658.6313612638, 267467.04320285696, 260796.654993712)

In [3]:
hook_RIH_0[0]-20000, hook_RIH_250[0]-20000, hook_RIH_470[0]-20000

(255481.1721150264, 237358.4852768013, 229877.088458336)

In [4]:
hook_ROB_0, hook_ROB_250, hook_ROB_470

(array([291120.47276499]), array([272389.70483966]), array([264941.80619539]))

Hookload Results - 17000 ft

In [6]:
hook_POOH_0[0]+20000, hook_POOH_250[0]+20000, hook_POOH_470[0]+20000

(350851.1199714261, 322481.57054550416, 312758.09837233427)

In [7]:
hook_RIH_0[0]-20000, hook_RIH_250[0]-20000, hook_RIH_470[0]-20000

(248356.405565285, 222379.9692879834, 210026.96793526382)

In [8]:
hook_ROB_0, hook_ROB_250, hook_ROB_470

(array([296347.48780942]), array([272923.75207587]), array([264401.00148752]))

In [4]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=total_lengths, y=hook_POOH_470+20000, name='POOH 470 gpm', line=dict(color='darkred', width=3, dash='dashdot')))
fig.add_trace(go.Scatter(x=total_lengths, y=hook_RIH_470-20000, name='RIH 470 gpm', line=dict(color='orange', width=3, dash='dashdot'))) 
fig.add_trace(go.Scatter(x=total_lengths, y=hook_ROB_470, name='ROB 470 gpm', line=dict(color='navy', width=3, dash='dashdot'))) 

fig.add_trace(go.Scatter(x=total_lengths, y=hook_POOH_250+20000, name='POOH 250 gpm', line=dict(color='darkred', width=3, dash='dash')))
fig.add_trace(go.Scatter(x=total_lengths, y=hook_RIH_250-20000, name='RIH 250 gpm', line=dict(color='orange', width=3, dash='dash'))) 
fig.add_trace(go.Scatter(x=total_lengths, y=hook_ROB_250, name='ROB 250 gpm', line=dict(color='navy', width=3, dash='dash'))) 

fig.add_trace(go.Scatter(x=total_lengths, y=hook_POOH_0+20000, name='POOH 0 gpm', line=dict(color='darkred', width=3, dash='dot')))
fig.add_trace(go.Scatter(x=total_lengths, y=hook_RIH_0-20000, name='RIH 0 gpm', line=dict(color='orange', width=3, dash='dot'))) 
fig.add_trace(go.Scatter(x=total_lengths, y=hook_ROB_0, name='ROB 0 gpm', line=dict(color='navy', width=3, dash='dot'))) 

# Update the layout for white background and improved readability
fig.update_xaxes(minor=dict(ticklen=6, tickcolor="black", showgrid=True))
fig.update_yaxes(minor_ticks="inside")
fig.update_layout(
    
    template= "none",
    title='Hookload vs. Bit Depth for Various GPM Rates',  # Add a title
    title_font_size=24,
    xaxis=dict(
        title='Bit Depth [ft]',
        titlefont_size=20,
        tickfont_size=14,
        tickmode = 'linear',
        tick0 = 13000,
        dtick = 500
    ),
    yaxis=dict(
        title='Hookload [lbf]',
        titlefont_size=20,
        tickfont_size=14,
        tickmode = 'linear',
        tick0 = 200000,
        dtick = 10000
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    # autosize=True,
    height = 500,
    width = 1000
)

# Show the figure
fig.show()
# fig.write_html("Plot.html")

Fluif Force Calculation

In [9]:
from lib.circulation import circl
import numpy as np
total_lengths = np.arange(12500, 18001, 100)


In [10]:
flu_470_13k = circl(flowrate=470, length=13000)
flu_250_13k = circl(flowrate=250, length=13000)
flu_470_17k = circl(flowrate=470, length=17000)
flu_250_17k = circl(flowrate=250, length=17000)
flu_250_total = [-circl(flowrate=250, length=i).sum() for i in total_lengths]
flu_470_total = [-circl(flowrate=470, length=i).sum() for i in total_lengths]


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig1 = make_subplots(rows=1, cols=2)

fig1.add_trace(go.Scatter(x=flu_470_13k, y=np.linspace(13000,0,len(flu_470_13k)), name='13k @ 470 gpm', line=dict(color='darkred', width=3, dash='dashdot')), row=1, col=1)
fig1.add_trace(go.Scatter(x=flu_250_13k, y=np.linspace(13000,0,len(flu_250_13k)), name='13k @ 250 gpm', line=dict(color='orange', width=3, dash='dashdot')), row=1, col=1)
fig1.add_trace(go.Scatter(x=flu_470_17k, y=np.linspace(17000,0,len(flu_470_17k)), name='17k @ 470 gpm', line=dict(color='navy', width=3, dash='dash')), row=1, col=2)
fig1.add_trace(go.Scatter(x=flu_250_17k, y=np.linspace(17000,0,len(flu_250_17k)), name='17k @ 250 gpm', line=dict(color='teal', width=3, dash='dash')), row=1, col=2)

# Update the layout

fig1.update_layout(
    
    template= "none",
    title='Fluid Force for Various GPM Rates',
    title_font_size=24,
    xaxis=dict(
        title='Fluid Force [lbf]',
        titlefont_size=20
    ),
    yaxis=dict(
        title='Bit Depth [ft]',
        titlefont_size=20,
        autorange="reversed"
    ),
    xaxis2=dict(
        title='Fluid Force [lbf]',
        titlefont_size=20
    ),
    yaxis2=dict(
        title='Bit Depth [ft]',
        titlefont_size=20,
        autorange="reversed"
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    height = 600,
    width = 1000
)

fig1.show()
# fig.write_html("Plot.html")

In [12]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=total_lengths, y=flu_470_total, name='Net Force @ 470 gpm', line=dict(color='slategrey', width=3, dash='dash')))
fig2.add_trace(go.Scatter(x=total_lengths, y=flu_250_total, name='Net Force @ 250 gpm', line=dict(color='orange', width=3, dash='dash')))

# Update the layout
fig2.update_layout(
    template= "none",
    title='Net Fluid Force for different Bit Depths',
    title_font_size=24,
    xaxis=dict(
        title='Bit Depth [ft]',
        titlefont_size=20
    ),
    yaxis=dict(
        title='Fluid Force [lbf]',
        titlefont_size=20
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    autosize=True,
)

fig2.show()

Restriction Check

In [1]:
from lib.tripping import trip_operation
import numpy as np

# Define the range of total lengths
total_lengths = [13000]
wob = 0

POOH_470 = np.array([trip_operation(F_b=wob,  flowrate=470, length=i, operation='POOH') for i in total_lengths])
RIH_470 = np.array([trip_operation(F_b=wob,  flowrate=470, length=i, operation='RIH') for i in total_lengths])
ROB_470 = np.array([trip_operation(F_b=wob,  flowrate=470, length=i, operation='ROB') for i in total_lengths])
POOH_250 = np.array([trip_operation(F_b=wob,  flowrate=250, length=i, operation='POOH') for i in total_lengths])
RIH_250 = np.array([trip_operation(F_b=wob,  flowrate=250, length=i, operation='RIH') for i in total_lengths])
ROB_250 = np.array([trip_operation(F_b=wob,  flowrate=250, length=i, operation='ROB') for i in total_lengths])
POOH_0 = np.array([trip_operation(F_b=wob,  flowrate=0, length=i, operation='POOH') for i in total_lengths])
RIH_0 = np.array([trip_operation(F_b=wob,  flowrate=0, length=i, operation='RIH') for i in total_lengths])
ROB_0 = np.array([trip_operation(F_b=wob,  flowrate=0, length=i, operation='ROB') for i in total_lengths])

c:\Users\User\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Web Extension extension is not supported and will be removed
  for idx, row in parser.parse():


In [1]:
247890-17660

230230

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, subplot_titles=("RIH", "ROB", "POOH"), shared_yaxes=True)

fig.add_trace(go.Scatter(x=RIH_470[0], y=np.linspace(13000, 0, len(POOH_470[0])), name='RIH 470 gpm', line=dict(color='darkred', width=3, dash='dashdot')), row=1, col=1)
fig.add_trace(go.Scatter(x=RIH_250[0], y=np.linspace(13000, 0, len(RIH_470[0])), name='RIH 250 gpm', line=dict(color='orange', width=3, dash='dashdot')), row=1, col=1) 
fig.add_trace(go.Scatter(x=RIH_0[0], y=np.linspace(13000, 0, len(ROB_470[0])), name='RIH 0 gpm', line=dict(color='navy', width=3, dash='dashdot')), row=1, col=1) 
fig.add_trace(go.Scatter(x=ROB_470[0], y=np.linspace(13000, 0, len(POOH_250[0])), name='ROB 470 gpm', line=dict(color='sienna', width=3, dash='dash')), row=1, col=2)
fig.add_trace(go.Scatter(x=ROB_250[0], y=np.linspace(13000, 0, len(RIH_250[0])), name='ROB 250 gpm', line=dict(color='sandybrown', width=3, dash='dash')), row=1, col=2) 
fig.add_trace(go.Scatter(x=ROB_0[0], y=np.linspace(13000, 0, len(ROB_250[0])), name='ROB 0 gpm', line=dict(color='seagreen', width=3, dash='dash')), row=1, col=2) 
fig.add_trace(go.Scatter(x=POOH_470[0], y=np.linspace(13000, 0, len(POOH_0[0])), name='POOH 470 gpm', line=dict(color='turquoise', width=3, dash='dot')), row=1, col=3)
fig.add_trace(go.Scatter(x=POOH_250[0], y=np.linspace(13000, 0, len(RIH_0[0])), name='POOH 250 gpm', line=dict(color='wheat', width=3, dash='dot')), row=1, col=3) 
fig.add_trace(go.Scatter(x=POOH_0[0], y=np.linspace(13000, 0, len(ROB_0[0])), name='POOH 0 gpm', line=dict(color='violet', width=3, dash='dot')), row=1, col=3) 

# Update the layout for white background and improved readability
fig.update_xaxes(minor=dict(ticklen=6, tickcolor="black", showgrid=True))
fig.update_yaxes(minor_ticks="inside")
fig.update_layout(
    
    template= "none",
    title='Axial Force vs. MD for Various GPM Rates',  # Add a title
    title_font_size=24,
    xaxis=dict(
        title='Axial Force [lbf]',
        titlefont_size=20,
        tickfont_size=14
    ),
    yaxis=dict(
        title='MD [ft]',
        titlefont_size=20,
        tickfont_size=14,
        autorange="reversed"
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    height = 600,
    width = 1400
)

# Show the figure
fig.show()
# fig.write_html("Plot.html")

Hookload Results - Closer to the Bit

In [5]:
POOH_0+20000, POOH_250+20000, POOH_470+20000

(327303.3890616717, 307422.5648260756, 294183.02531721187)

In [6]:
RIH_0-20000, RIH_250-20000, RIH_470-20000

(255163.46029075357, 237260.4386971984, 224752.63135406972)

In [9]:
ROB_0, ROB_250, ROB_470

(array(290487.82797127), array(272026.74268909), array(259563.72835863))

Hookload Results - Further from the bit

In [3]:
POOH_0+20000, POOH_250+20000, POOH_470+20000

(327302.3503764293, 307782.69388694846, 295836.66643494536)

In [4]:
RIH_0-20000, RIH_250-20000, RIH_470-20000

(255186.13271226687, 237655.2076077912, 226458.80496666842)

In [5]:
ROB_0, ROB_250, ROB_470

(array(290497.96624531), array(272401.32755323), array(261231.60225132))

In [18]:
import plotly.graph_objects as go

trips = ['RIH', 'ROB', 'POOH']
fig = go.Figure()

fig.add_trace(go.Bar(
    x=trips,
    y=[235755, 270835, 305770],
    name='Normal',
    marker_color='indianred',
    text=[235755, 270835, 305770],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=trips,
    y=[224752, 259564, 294183],
    name='Restriction',
    marker_color='lightsalmon',
    text=[224752, 259564, 294183],
    textposition='auto'
))

fig.update_layout(
    barmode='group', 
    template= "none",
    title='Hookload with and without restriction @ 470 gpm',
    title_font_size=24,
    xaxis=dict(
        titlefont_size=20
    ),
    yaxis=dict(
        title='Hookload [lbf]',
        titlefont_size=20
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    height=600, 
    width=800
    )

fig.show()

In [31]:
import plotly.graph_objects as go

trips = ['RIH', 'ROB', 'POOH']
fig = go.Figure()

fig.add_trace(go.Bar(
    x=trips,
    y=[239923, 274964, 310738],
    name='Normal',
    marker_color='#1D2D44',
    text=[239923, 274964, 310738],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=trips,
    y=[237655, 272401, 307782],
    name='Restriction',
    marker_color='#748CAB',
    text=[237655, 272401, 307782],
    textposition='auto'
))

fig.update_layout(
    barmode='group', 
    template= "none",
    title='Hookload with and without restriction @ 250 gpm',
    title_font_size=24,
    xaxis=dict(
        titlefont_size=20
    ),
    yaxis=dict(
        title='Hookload [lbf]',
        titlefont_size=20
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    height=600, 
    width=800
    )

fig.show()